In [73]:
import os
import pandas as pd
import json

In [74]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### 데이터 불러오기

#### 1. 가사

In [75]:
lyrics1 = pd.read_csv("./dataset/label_result_song_short.csv")
lyrics2 = pd.read_csv("./dataset/translated_lyrics.csv")

lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

#### 2. 소설

In [76]:
novel_final = pd.read_csv('./dataset/classified_novel1.csv')
data = novel_final.copy()

In [77]:
# data = data.sample(n=50000, random_state=42).reset_index(drop=True)

len(data)

885014

In [78]:
data.head()

,Column1,file_id,title,paragraph_id,text,labels,sentiment
0,0,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.1,01범보다 무서운 곶감,4.0,불안
1,1,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.2,화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로...,4.0,불안
2,2,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.3,그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 ...,2.0,분노
3,3,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.4,"그래 옛날, 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성...",4.0,불안
4,4,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.5,"애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가, 응? 그 집 문...",1.0,슬픔


### Preprocessing

#### 1. 가사

In [79]:
# 감정 카테고리
emotions = ['상처', '불안', '기쁨', '슬픔', '분노', '당황']

In [80]:
# 각 id에 대해 sentiment_label의 빈도를 계산하여, 가장 많이 나온 2개의 감정을 새로운 칼럼으로 추가

def get_top_two_sentiments(sentiments):
    sentiment_counts = sentiments.value_counts()
    # 가장 많은 2개 감정을 반환
    top_two = sentiment_counts.head(2).index.tolist()
    # 2개 미만일 경우 빈 값 처리
    return top_two + [None] * (2 - len(top_two))

# groupby와 agg를 활용해 변환
lyrics_final = lyrics.groupby("id").agg({
    "title": "first",   # 첫 번째 값 유지
    "singer": "first",  # 첫 번째 값 유지
    "genres": "first",  # 첫 번째 값 유지
    "lyrics": "first",  # 첫 번째 값 유지
    "translated_lyrics": list,  # 리스트로 묶음
    "sentiment_label": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment_label에서 두 개의 감정을 각각의 칼럼으로 분리
lyrics_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(lyrics_final['sentiment_label'].tolist(), index=lyrics_final.index)

# 불필요한 sentiment_label 열 삭제
lyrics_final = lyrics_final.drop(columns=["sentiment_label"])

In [81]:
lyrics_final.head()

,id,title,singer,genres,lyrics,translated_lyrics,top_sentiment,second_sentiment
0,418168,희재,성시경,"발라드, 국내영화",햇살은 우릴 위해 내리고,"[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 ...",슬픔,기쁨
1,418598,친구라도 될 걸 그랬어,거미 (GUMMY),R&B/Soul,벌써 넌 내가 편하니,"[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는...",불안,슬픔
2,711626,살다가,SG 워너비,발라드,살아도 사는 게 아니래,"[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게...",슬픔,불안
3,1500196,내사람,SG 워너비,R&B/Soul,내 가슴속에 사는 사람 내가 그토록 아끼는 사람,"[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도...",기쁨,불안
4,1854856,라라라,SG 워너비,발라드,그대는 참 아름다워요,"[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감...",기쁨,슬픔


In [82]:
le = LabelEncoder()
le.fit(emotions)

lyrics_final['감정1_encoded'] = le.transform(lyrics_final['top_sentiment'])
lyrics_final['감정2_encoded'] = le.transform(lyrics_final['second_sentiment'])

#### 2. 소설

In [83]:
print("NaN 개수:", data['text'].isna().sum())  # NaN 개수 확인
print("None 포함 여부:", data['text'].isnull().sum())  # None 개수 확인

NaN 개수: 79
None 포함 여부: 79


In [84]:
data['text'] = data['text'].fillna("")

print("NaN 개수:", data['text'].isna().sum())  # NaN 개수 확인
print("None 포함 여부:", data['text'].isnull().sum())  # None 개수 확인

NaN 개수: 0
None 포함 여부: 0


In [85]:
# 같은 title을 가진 데이터에서 100개씩 묶어 그룹화
data["group"] = data.groupby("title").cumcount() // 100  # 100개 단위 그룹 생성

In [86]:
data = data.groupby(["file_id", "group"]).agg({
    "title": "first",  # 대표 file_id
    "text": list,  # 100개씩 묶음
    "sentiment": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment에서 두 개의 감정을 각각의 칼럼으로 분리
data[['top_sentiment', 'second_sentiment']] = pd.DataFrame(data['sentiment'].tolist(), index=data.index)

# 불필요한 sentiment 칼럼 삭제
data = data.drop(columns=["sentiment", "group", "file_id"])

data.head()

,title,text,top_sentiment,second_sentiment
0,이야기꾼 구연설화,"[01범보다 무서운 곶감, 화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 ...",분노,당황
1,이야기꾼 구연설화,"[<봄꿩은 제 울음에 저 죽는다>, 그 말과 같아서 사램이 잘못 되머넌 하는 말여....",상처,분노
2,이야기꾼 구연설화,"[그래 그 여자가 쪄서 쌀얼 쪄서 밥얼 했어., “잡수라.”구., 그래 한 때를 몽...",상처,당황
3,이야기꾼 구연설화,"[그랴. 그래 우리 인제 사춘 찾어간다는 얘기를 족~ 갈쳐중개,, “그러시냐구. 그...",상처,분노
4,이야기꾼 구연설화,"[나와서 인제 그 집 먼지 인저 그, 그, 뭐여 면사무소 있는 디 나와서 인제, 거...",상처,분노


In [87]:
print("NaN 개수:", data['text'].isna().sum())  # NaN 개수 확인
print("None 포함 여부:", data['text'].isnull().sum())  # None 개수 확인

NaN 개수: 0
None 포함 여부: 0


In [88]:
# 'No'를 감정 목록에 추가하여 훈련 데이터에 포함
emotions_with_no = list(emotions) + ['No']

# LabelEncoder 초기화 후 감정 목록에 'No' 포함하여 학습
le = LabelEncoder()
le.fit(emotions_with_no)

# 감정1과 감정2를 숫자로 변환
data['감정1_encoded'] = le.transform(data['top_sentiment'])
data['감정2_encoded'] = le.transform(data['second_sentiment'])

# 수치형 감정1, 감정2와 텍스트 결합
data['input_text'] = data['감정1_encoded'].astype(str) + " | " + data['감정2_encoded'].astype(str) + " | " + data['text'].astype(str)

# 불필요한 문자 제거
data['input_text'] = data['input_text'].str.replace(r"[\[\],']", "", regex=True)

data[['top_sentiment', 'second_sentiment', 'input_text']].sample(n=5)


,top_sentiment,second_sentiment,input_text
2455,분노,기쁨,"3 | 1 | ""사랑해."" 멍…… 정말 멍…… 얘가 지금 뭐라는 거야? ""너 지금…..."
6203,불안,상처,4 | 5 | 으뜸과 곧음이 시작되었다 끝났다 한다 진실로 이러한 이치를 밝게 안다...
315,상처,분노,5 | 3 | “그건 그럴 수밖에 없었어요. 해원이가 은밀하게 숨긴 데에는 그만 한...
1934,불안,슬픔,4 | 6 | 그녀는 K와 손을 잡고 오솔길을 따라 오르며 나누었던 옛 이야기에 몰...
1046,불안,기쁨,"4 | 1 | ""나중에 신혼여행 갔다 오면 한 번 만나자."" ""그래."" ""일……다시..."


In [89]:
len(data)

8972

In [90]:
# 감정별 숫자 매핑 출력
for emotion, label in zip(le.classes_, le.transform(le.classes_)):
    print(f"{emotion}: {label}")

No: 0
기쁨: 1
당황: 2
분노: 3
불안: 4
상처: 5
슬픔: 6


In [91]:
# 데이터셋 클래스 정의
class NovelsDataset(Dataset):
    def __init__(self, data, tokenizer, text_column="input_text", max_length=128):
        self.data = data.reset_index(drop=True)  # 인덱스 리셋 (중복 방지)
        self.tokenizer = tokenizer
        self.text_column = text_column
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.loc[idx, self.text_column] # input_text 가져오기

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        # labels 생성 (input_ids 복사 및 패딩 토큰을 -100으로 설정)
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        }

### Modeling

In [92]:
# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

# 모델 초기화
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.bos_token_id = tokenizer.bos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [93]:
dataset = NovelsDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=2)

In [94]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results/emotion',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    save_steps=200,
    save_total_limit=2,
    logging_steps=50,
    eval_strategy="epoch",
)

In [95]:
train_novels, eval_novels = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = NovelsDataset(train_novels, tokenizer)
eval_dataset = NovelsDataset(eval_novels, tokenizer)

In [96]:
# Trainer에 데이터셋 전달
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 검증 데이터셋
)

In [97]:
sample = train_dataset[0]
print("Input IDs:", sample['input_ids'])
print("Attention Mask:", sample['attention_mask'])
print("Labels:", sample['labels'])

Input IDs: tensor([ 9130,   739,   466,  9045,   739,   466, 10063,  7584,  9391, 21182,
        12311, 35142, 13875,  9317, 16299, 13568, 18017,  9717, 17631, 11247,
        15972,  7621,  7422,  8263,   739,     5, 23775, 47711, 10063,  7669,
         7445, 11001, 22580, 26309,  7185,  9135, 13115,   406,   377,  9546,
         7407, 32552,  9069,  7560,  8159, 22301,  8024,  9274,  7407,  7543,
         8146, 16626, 13872, 10063,  8762, 10072,  9215, 18282,  9215,  9878,
        13969,  9261, 19023,  9174,  9179,  9432, 12004,  9825, 12024,  7281,
         6883,  9173, 13444, 32987,   377, 10063,  8052,  9055,  8694, 15509,
          376,  9174,  9179,  9432, 17104,  8744,  7055,  7652,  6919,  7514,
          389, 46651,  6824, 12222, 21832,  9122,  7991,   406, 25758,  9720,
         6958,  9237, 22882,   406, 16518, 15518, 39717,  9193, 22882,   406,
          377,  9546,  7407, 19902, 18191,  6824, 32738,  8006,  9042,  9313,
        44083, 14870,  9064,  7472,  7326,  9016, 100

In [98]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss
0,4.139200,4.040720
1,3.815300,4.004372
2,3.551100,4.024973
3,3.310000,4.042984
4,3.123900,4.064175


TrainOutput(global_step=2520, training_loss=3.593004034435938, metrics={'train_runtime': 1067.9079, 'train_samples_per_second': 37.803, 'train_steps_per_second': 2.36, 'total_flos': 2634868850688000.0, 'train_loss': 3.593004034435938, 'epoch': 4.998761456527124})

In [68]:
device = torch.device("cuda")

In [69]:
# 이야기 생성 함수
def generate_story(lyrics_input):
    # 입력 토큰화
    input_ids = tokenizer.encode(lyrics_input, return_tensors='pt').to(device)

    # 출력 생성
    output = model.generate(
        input_ids, 
        max_new_tokens=150, # 새로 생성할 토큰의 개수를 제한
        num_return_sequences=1, 
        temperature=0.8, 
        top_k=50, 
        top_p=0.9, 
        repetition_penalty=1.2, 
        do_sample=True  # 샘플링 활성화
    )

    # 입력 길이 추적
    input_length = input_ids.shape[1]

    # 생성된 토큰 중 입력 토큰 이후의 부분만 디코딩
    generated_tokens = output[0][input_length:]
    generated_story = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_story

In [119]:
i = 725
print(lyrics_final["title"][i])

밤양갱


In [120]:
lyrics = " ".join(lyrics_final["translated_lyrics"][i])

prompt = lyrics_final['감정1_encoded'][i].astype(str) + " | " + lyrics_final['감정2_encoded'][i].astype(str) + " | " + lyrics
print(prompt)

2 | 5 | 떠나는 길에 니가 내게 말했지 ‘너는 바라는 게 너무나 많아 잠깐이라도 널 안 바라보면 머리에 불이 나버린다니까’ 나는 흐르려는 눈물을 참고 하려던 얘길 어렵게 누르고 ‘그래 미안해’라는 한 마디로 너랑 나눈 날들 마무리했었지 달디달고 달디달고 달디단 밤양갱 밤양갱 내가 먹고 싶었던 건 달디단 밤양갱 밤양갱이야 떠나는 길에 니가 내게 말했지 ‘너는 바라는 게 너무나 많아’ 아냐 내가 늘 바란 건 하나야 한 개뿐이야 달디단 밤양갱 달디달고 달디달고 달디단 밤양갱 밤양갱 내가 먹고 싶었던 건 달디단 밤양갱 밤양갱이야 상다리가 부러지고 둘이서 먹다 하나가 쓰러져버려도 나라는 사람을 몰랐던 넌 떠나가다가 돌아서서 말했지 ‘너는 바라는 게 너무나 많아’ 아냐 내가 늘 바란 건 하나야 한 개뿐이야 달디단 밤양갱


In [121]:
# 가사로 이야기 생성
generated_story = generate_story(prompt)
print("생성된 이야기:", generated_story)

생성된 이야기: 이 다람쥐처럼 그 사람이 내 곁에 머물러 있었어. 하지만 시간이 흘러 이제 내 곁을 떠난다는 걸 모르고 흘려 보낸 날들이 후회하는 날들을 한탄하며 이대로 그리움에 허탈한 마음으로 그리워했던 시간만 보내며 아쉬움이 눈물로 잠들어가는 동안 내가 간절함을 달그락거리지 못해 안타까워하는 걸 어찌할 뿐.... “니 너는 내게 그런 애틋함이나 절망을 참을 담아두고 나를 위해 네 가슴속히 간직하리라 할 수놓는구나~ 하고 속절없이 기다릴 줄 알면 나도 모르게 해놓고 갈망나니 울어버림 같은 마음을 채워다오.” 하며 너를 기다리는 동안 나의 사랑하더니 이제 남은 이야기들은 이제 잊고 싶어서 슬프게 해놓았


In [ ]:
# 가사로 이야기 생성
# lyrics_input = ""
# generated_story = generate_story(lyrics_input, tag1="2", tag2="2")
# print("생성된 이야기:", generated_story)

생성된 이야기: 인지 이젠데 왜 내가 좋아하는지 그래 속상하기만 말예 퇴근거리는 게 참으론 정말이지 남이 나를 버리고 지금이 딱히 한숨 쉬었는지 알면서.... 이러면 금방엔 숨소리 질러 가며 소리 죽여 버릇한 기분이 좋아서 그냥 그러는 거냐? 이제 더러운 세상 물끄떡하고 예사로운 걸어서 그런 건가 못해~ 하고 사는데. 그만 이렇게 예쁜 척하는 소리가 나랑말루 오죽 나래 그래 가지 않고 그저 눈 감고 삐딱다리 놓으면 살그렁거니 이마에 이어두 못 하게 할까 봐놓고 사는 저러다 말고 다리 뻗고 가만히 들여다보면 예뻐한다ᆞ오래야 제자리 앉으며 맘


In [122]:
# 생성된 이야기들을 저장할 리스트
generated_stories = []

# 701부터 750까지 반복하여 가사로 이야기 생성
for i in range(701, 751):
    # 각 가사 제목과 번역된 가사 추출
    title = lyrics_final["title"][i]
    prompt = " ".join(lyrics_final["translated_lyrics"][i])

    # 가사로 이야기 생성
    generated_story = generate_story(prompt)

    # 결과 저장
    generated_stories.append({"title": title, "generated_story": generated_story})

# 결과를 데이터프레임으로 변환
df = pd.DataFrame(generated_stories)

# CSV로 저장
df.to_csv("emotion-tagged-generated_stories.csv", index=False)

print("csv 저장 완료")


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
